In [2]:
%run demo.py /data/tushar/datasets/2011_09_26/2011_09_26_drive_0001_sync/

Processing: Frame: 0
.... failed!
Processing: Frame: 1
412
[998.0, 38.0]
Matches: 281 Inliers: 55.16014234875445 %, Current pose:
   0.9999974   -0.0009507    0.0020530   -0.0019343 
   0.0009527    0.9999991   -0.0009711    0.0007011 
  -0.0020521    0.0009730    0.9999974    1.3557623 
   0.0000000    0.0000000    0.0000000    1.0000000 
Processing: Frame: 2
420
[983.0, 238.0]
Matches: 279 Inliers: 58.064516129032256 %, Current pose:
   0.9999899   -0.0009850    0.0043873   -0.0042179 
   0.0009990    0.9999944   -0.0031866   -0.0030875 
  -0.0043841    0.0031909    0.9999853    2.7224062 
  -0.0000000    0.0000000    0.0000000    1.0000000 
Processing: Frame: 3
383
[1026.0, 185.0]
Matches: 294 Inliers: 55.78231292517007 %, Current pose:
   0.9999772   -0.0006713    0.0067187    0.0027070 
   0.0007006    0.9999903   -0.0043500   -0.0028913 
  -0.0067157    0.0043546    0.9999680    4.0931759 
  -0.0000000   -0.0000000   -0.0000000    1.0000000 
Processing: Frame: 4
399
[993.0, 244.0

In [31]:
import viso2
import numpy as np
import matplotlib.pyplot as plt
#from mayavi import mlab
from skimage.io import imread
import pathlib as pl
import sys
from tqdm import tqdm
import cv2

In [2]:
def filter_by_name(name, imgs):
    return sorted(filter(lambda x: str(x).find(name) >= 0, imgs))

In [ ]:
LEFT="image_00"
RIGHT="image_01"
base_dir = pl.Path("/data/tushar/datasets/2011_09_26/2011_09_26_drive_0002_sync")

In [23]:
LEFT="image_2"
RIGHT="image_3"
base_dir = pl.Path("/data/tushar/datasets/dataset/sequences/00")

In [24]:
# set the most relevant parameters
params = viso2.Stereo_parameters()
# params.calib.f  = 721.5377
# params.calib.cu = 609.5593
# params.calib.cv = 172.854
# params.base     = 0.537

params.calib.f  = 718.856
params.calib.cu = 607.1928
params.calib.cv = 185.2157
params.base     = 0.5323318472000029

#initialize visual odometry 
viso = viso2.VisualOdometryStereo(params)
recon = viso2.Reconstruction()
recon.setCalibration(params.calib.f, params.calib.cu, params.calib.cv)
matcher_params = viso2.Matcher_parameters()

matcher_params.f = 718.856
matcher_params.cu = 607.1928
matcher_params.cv = 185.2157
matcher_params.base = 0.5323318472000029
matcher_params.nms_n = 13
matcher_params.nms_tau = 50
matcher_params.match_binsize = 50 
matcher_params.match_radius = 200
matcher_params.match_disp_tolerance = 2
matcher_params.outlier_flow_tolerance = 10
matcher_params.outlier_disp_tolerance = 10
matcher_params.multi_stage = 1
matcher_params.half_resolution = 0
matcher_params.refinement = 1

In [25]:
matcher = viso2.Matcher(matcher_params)
matcher.setIntrinsics(params.calib.f, params.calib.cu, params.calib.cv, params.base)

In [26]:
all_images = [f for f in base_dir.rglob("*.png")]
left_gray = filter_by_name(LEFT, all_images)
right_gray = filter_by_name(RIGHT, all_images)

In [27]:
N = len(left_gray)
assert(len(left_gray) == len(right_gray))

In [29]:
def im2Gray(pilImage):    
    img = cv2.cvtColor(np.array(pilImage), cv2.COLOR_BGR2GRAY)
    return img

In [32]:
pose = viso2.Matrix_eye(4)
poses = []
for i in tqdm(range(N),ncols=80):
    # load the images
    left_img = im2Gray(imread(str(left_gray[i])))
    right_img = im2Gray(imread(str(right_gray[i])))
    assert(len(left_img.shape) == 2) # should be grayscale
    
    #print("Processing: Frame:", i)

    matcher.pushBack(left_img, right_img)
    
    if i > 0:
        matcher.matchFeatures(2)
        matches = matcher.getMatches()
        matches_mat = np.empty([8, matches.size()])
        #print(matches.size())

        for i,m in enumerate(matches):
            if i == 342:
                #print("{}".format([m.u1p, m.v1p]))
                matches_mat[:, i] = (m.u1p, m.v1p, m.u2p, m.v2p,m.u1c,m.v1c,m.u2c,m.v2c)
        
    if viso.process_frame(left_img, right_img):
        motion = viso.getMotion()
        est_motion = viso2.Matrix_inv(motion)
        pose = pose * est_motion

        num_matches = viso.getNumberOfMatches()
        num_inliers = viso.getNumberOfInliers()
        #print('Matches:', num_matches, "Inliers:", 100*num_inliers/num_matches, '%, Current pose:')
        #print(pose)
        poses.append(np.array([float(i) for i in str(pose).split()]).reshape(4,4))

        matches = viso.getMatches()
        matches_mat = np.empty([2, matches.size()])
      
        matches_mat

        assert(matches.size() == num_matches)
        recon.update(matches, motion, 0)
    else:
        #print('.... failed!')
        pass

100%|███████████████████████████████████████| 4541/4541 [07:39<00:00,  9.87it/s]


In [33]:
points = recon.getPoints()
print("Reconstructed", points.size(), "points...")

Reconstructed 168036 points...


In [34]:
pts = np.empty((points.size(),3))
for i,p in enumerate(points):
    pts[i,:] = (p.x, p.y, p.z)

In [35]:
trajectory = np.array([list(pose[0:3,3]) for pose in poses])

In [36]:
import plotly_express as px
from plotly.offline import plot 
fig = px.scatter_3d(trajectory, x=trajectory[:,0], y=trajectory[:,1], z=trajectory[:,2])
plot(fig,filename="../results/odom_kitti_seq_00_svo.html")

'../results/odom_kitti_seq_00_svo.html'

In [15]:
trajectory

array([[ 3.58310000e-03,  1.27301000e-02,  8.91110100e-01],
       [ 3.36140000e-03,  1.40025000e-02,  1.78247430e+00],
       [-2.48880000e-03,  2.05002000e-02,  2.70598580e+00],
       [ 9.58110000e-03,  2.09564000e-02,  3.61692070e+00],
       [ 1.65845000e-02,  3.02862000e-02,  4.51731770e+00],
       [ 1.46542000e-02,  3.87561000e-02,  5.41869240e+00],
       [ 2.17037000e-02,  4.86174000e-02,  6.35882080e+00],
       [ 3.02855000e-02,  5.27031000e-02,  7.30926200e+00],
       [ 2.41441000e-02,  5.68596000e-02,  8.24004870e+00],
       [ 1.91713000e-02,  6.24112000e-02,  9.17624850e+00],
       [ 1.33676000e-02,  5.67535000e-02,  1.01139996e+01],
       [-6.42450000e-03,  6.97173000e-02,  1.10506982e+01],
       [-2.39525000e-02,  7.39659000e-02,  1.19865722e+01],
       [-5.48177000e-02,  8.33036000e-02,  1.29236441e+01],
       [-9.70381000e-02,  8.71894000e-02,  1.38604636e+01],
       [-1.47103700e-01,  9.08642000e-02,  1.48027618e+01],
       [-2.00015300e-01,  9.78558000e-02